In [1]:
import pandas as pd

In [2]:
import sqlite3

In [3]:
import os

In [4]:
        # create the sqlite DB
db = sqlite3.connect("Recalls.db")

In [5]:
df = pd.read_excel(r".\FSIS-Recall-Summary-2014.xlsx", header=1, index =2)

        #convert pandas to SQL table
df.to_sql("recalls", db, if_exists="replace")

def run_query(query):
    return pd.read_sql_query(query,db)

os.getcwd

C:\Users\Work\Anaconda3\lib\site-packages\pandas\core\generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


<function nt.getcwd()>

In [6]:
query = """
SELECT * 
FROM sqlite_master
ORDER BY name;
"""
run_query(query)

# df2 = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
# df2     # -- EXAMPLE OF WHAT CAN BE DONE IF NOT USING A FUNCTION --

,type,name,tbl_name,rootpage,sql
0,index,ix_recalls_index,recalls,4,"CREATE INDEX ""ix_recalls_index""ON ""recalls"" (""..."
1,table,recalls,recalls,2,"CREATE TABLE ""recalls"" (\n""index"" INTEGER,\n ..."


In [7]:
# df3 = pd.read_sql('SELECT * FROM recalls', db)
# df3   # -- Better to assign new value for each query --

query = """
SELECT * 
FROM recalls;
"""
run_query(query)

,index,Recall Date,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled
0,0,2014-01-10 00:00:00,001-2014,I,Mechanically Separated Chicken Products,Salmonella,33840
1,1,2014-01-13 00:00:00,002-2014,I,Various Beef Products,Other,42103
2,2,2014-01-15 00:00:00,003-2014,I,Beef Franks,Undeclared Allergen,2664
3,3,2014-01-17 00:00:00,004-2014,II,Beef and Pork Products,Undeclared Allergen,130000
4,4,2014-01-17 00:00:00,005-2014,I,Spiral Hams,Listeria monocytogenes,67113
...,...,...,...,...,...,...,...
89,89,2014-12-27 00:00:00,090-2014,I,Pork products,Undeclared Allergen,83666
90,90,2014-12-27 00:00:00,091-2014,I,Chicken and Beef Products,Undeclared Allergen,55813
91,91,2014-12-29 00:00:00,092-2014,I,Beef and Pork Products,Undeclared Allergen,38400
92,92,2014-12-31 00:00:00,093-2014,I,Chicken Products,Undeclared Allergen,5300


In [8]:
# df2 = pd.read_sql('SELECT * FROM recalls WHERE `Reason for Recall` = "Salmonella" ORDER BY `Pounds Recalled` DESC', db)
# df2

query = """
SELECT * 
FROM recalls
WHERE `Reason for Recall` = 'Salmonella'
ORDER BY `Pounds Recalled` DESC;
"""
run_query(query)

,index,Recall Date,Recall Number,Recall Class,Product,Reason for Recall,Pounds Recalled
0,14,2014-02-15 00:00:00,015-2014,I,Dried Egg Products,Salmonella,309594
1,0,2014-01-10 00:00:00,001-2014,I,Mechanically Separated Chicken Products,Salmonella,33840
2,72,2014-10-24 00:00:00,073-2014,I,Chicken Products,Salmonella,28980
3,43,2014-07-03 00:00:00,044-2014,I,Chicken Products,Salmonella,0


In [12]:
query = """
SELECT SUM(`Pounds Recalled`) AS "Total Pounds Recalled" 
FROM recalls;
"""
run_query(query)

,Total Pounds Recalled
0,18675102


In [14]:
query = """
SELECT `Reason for Recall`, 
COUNT(*) AS "Number of Cases"
FROM recalls
GROUP BY `Reason for Recall`
ORDER BY COUNT(*) DESC
LIMIT 5;
"""
run_query(query)

,Reason for Recall,Number of Cases
0,Undeclared Allergen,43
1,Other,23
2,Listeria monocytogenes,7
3,Extraneous Material,6
4,Salmonella,4
